In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras.datasets import cifar10

from tensorflow import device
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.layers import Dense,Conv2D,Activation,BatchNormalization,MaxPooling2D,Dropout,Flatten,experimental,Input,concatenate,GlobalAveragePooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K 

In [ ]:
data_set = "dogs-vs-cats"
import zipfile 
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/train.zip","r") as z:
    z.extractall(".")
    # save all files to kaggle/files/images
    destination = '/kaggle/files/images/train'
    z.extractall(destination)
    
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/test1.zip","r") as z:
    z.extractall(".")
    # save all files to kaggle/files/images
    destination = '/kaggle/files/images/test'
    z.extractall(destination)
def list_full_paths(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

train = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/train/train')})
train['truth_label'] = np.where(train['filepath'].str.contains('dog'), 'dog', 'cat')

test = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/test/test1')})

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(train, test_size=0.2,random_state = 10)

In [ ]:
img_shape = 224
ks = 16
drop_size = 0.1
act = "relu"
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 10:
        lrate = 0.0005
    if epoch > 20:
        lrate = 0.00025
    if epoch > 30:
        lrate = 0.0001
    return lrate

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    
    rotation_range=35,
    horizontal_flip=True,
    width_shift_range=0.25,
    height_shift_range=0.25,
    rescale = 1./255,
    zoom_range =0.3
    
    )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(dataframe=X_train, x_col='filepath', y_col='truth_label', class_mode='categorical', target_size = (img_shape, img_shape), batch_size = 32)
val_set = test_datagen.flow_from_dataframe(dataframe=X_val, x_col='filepath', y_col='truth_label', class_mode='categorical', target_size = (img_shape, img_shape), batch_size = 32)

In [ ]:
test = test_datagen.flow_from_dataframe(dataframe=test,
    directory = '/kaggle/files/images/test',
    x_col = 'filepath',
    y_col = None,
    class_mode = None,
    target_size = (img_shape, img_shape),
    batch_size = 32,
    shuffle = False)

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
model= ResNet50(weights='imagenet',
                   include_top=False,
                   pooling = "max",
                  input_shape=(224,224,3))
x = Dense(2048,activation = act)(model.output)
x = BatchNormalization()(x)
x = Dropout(drop_size*4)(x)
x = Dense(2048,activation = act)(x)
x = BatchNormalization()(x)
x = Dropout(drop_size*4)(x)
x = Dense(2,activation = "softmax")(x)
model = Model(model.input,outputs = x)
model.compile(optimizer = "adam",
                     loss = "binary_crossentropy",
                     metrics = ["accuracy"])

for layer in model.layers[:-8]:
    layer.trainable = False
for layer in model.layers[-8:]:
    layer.trainable = True

In [ ]:
model.summary()

In [ ]:
history = model.fit(training_set,validation_data = val_set,callbacks=[LearningRateScheduler(lr_schedule)],epochs = 40)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# test_preds = model.predict(test, steps = np.ceil(12500 /32))

# test["test_preds"] = np.argmax(test_preds, axis = 1)
# labels = dict((v,k) for k,v in training_set.class_indices.items())

# test['test_preds'] = test['test_preds'].map(labels)